In [1]:
from langchain_aws import ChatBedrockConverse
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph, START
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState

from typing import List
from typing_extensions import TypedDict

from typing import Literal

In [ ]:
llm = ChatBedrockConverse(
    model_id="us.amazon.nova-micro-v1:0",
    region_name='us-west-2',
    temperature=0.3
    )

In [3]:
retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id="KDVXCDF1BK",  # From your Bedrock Knowledge Base
    retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 2}},
     region_name="us-west-2"
)

C:\Users\Vishwas\AppData\Local\Temp\ipykernel_5968\4007869504.py:1: LangChainDeprecationWarning: The class `AmazonKnowledgeBasesRetriever` was deprecated in LangChain 0.3.16 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import AmazonKnowledgeBasesRetriever``.
  retriever = AmazonKnowledgeBasesRetriever(


In [4]:
def Foodagent(state):
    query= state['question']
    context= state['context']
    system_prompt = open("FOOD_agent.txt").read()
    
    messages = [
            (
                  "system",
                  system_prompt
            ),
            ("human",  f"""query: {query} /n/n context:{context}"""),
      ]
    ai_msg = llm.invoke(messages)
    return {
        "Foodagent_response": ai_msg.content
        }


In [5]:
def CGMagent(state):
      query= state['question']
      context= state['context']
      system_prompt = open("CGM_agent.txt").read()
      messages = [
            (
                  "system",
                  system_prompt
            ),
            ("human",  f"""query: {query} /n/n context:{context}"""),
      ]
      ai_msg = llm.invoke(messages)
      return {
        "CGMagent_response": ai_msg.content
        }


In [6]:
def IOTagent(state):
    query= state['question']
    context= state['context']
    system_prompt = open("IOT_agent.txt").read()
    messages = [
            (
                  "system",
                  system_prompt
            ),
            ("human",  f"""query: {query} /n/n context:{context}"""),
      ]
    ai_msg = llm.invoke(messages)
    return {
        "IOTagent_response": ai_msg.content
        }


In [13]:
len(open("MULTIOMICS_agent.txt").read())

16509

In [7]:
def MULTIOMICSagent(state):
    query= state['question']
    context= state['context']
    system_prompt= open("MULTIOMICS_agent.txt").read()
    messages = [
            (
                  "system",
                  system_prompt
            ),
            ("human",  f"""query: {query} /n/n context:{context}"""),
      ]
    ai_msg = llm.invoke(messages)
    return {
        "MULTIOMICSagent_response": ai_msg.content
        }


In [8]:
def combiner(state):
    context = "\n".join([
        state.get("Foodagent_response", ""),
        state.get("CGMagent_response", ""),
        state.get("IOTagent_response", ""),
        state.get("MULTIOMICSagent_response", ""),
    ])
    query = state['question']
    system_prompt= open("COMBINED_agent.txt").read()
    messages = [
            (
                  "system",
                  system_prompt
            ),
            ("human",  f"""query: {query} /n/n context:{context}"""),
      ]
    ai_msg = llm.invoke(messages)
    return {"combined_context": ai_msg.content}

In [ ]:
GRADE_PROMPT = """
You are a query routing agent based on agentic knowledge.
Your task is to select all applicable agents from the list below that are needed to answer the given user question.

Available Agents:
- FOOD_AGENT: Handles queries about food, diet, nutrition, meal planning.
- CGM_AGENT: Handles queries about continuous glucose monitoring (CGM), blood glucose, insulin levels.
- IOT_AGENT: Handles queries about IoT devices, wearable sensors, connected health devices.
- MULTIOMICS_AGENT: Handles queries about genomics, proteomics, microbiome, metabolomics, or other omics data.

A query can belong to more than one agent category.

Here is the user question:
{question}

Return a list of agent names from the list above that are best suited to answer this question.
If there are no agents that are applicable, return both MULTIOMICS_AGENT and IOT_AGENT.
"""
class RouteQuery(BaseModel):
      agents: list[Literal["FOOD_AGENT", "CGM_AGENT", "IOT_AGENT", "MULTIOMICS_AGENT"]] = Field(
        ...,
        description="List of agents relevant to the user question.")

def Routingagent(state):

    query = state['question']
    # generation = state['generation']
    prompt = GRADE_PROMPT.format(question=query)

    results = retriever.invoke(query)
    context = ""
    for doc in results:
      context += doc.page_content
    # state['context'] = context
    response = (
        llm
        # highlight-next-line
        .with_structured_output(RouteQuery).invoke(
            [{"role": "user", "content": prompt}]
        )
    )
    return {"agents_to_call": response.agents,"context":context}

In [10]:
Routingagent({"question": "in the trait category what are the SNPs involved in pigmentation and give a brief description for each of these SNPs."})

ValidationException: An error occurred (ValidationException) when calling the Converse operation: This model doesn't support tool use.

In [11]:


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
    """
    question: str
    Foodagent_response: str
    CGMagent_response: str
    IOTagent_response: str
    MULTIOMICSagent_response: str
    context: str
    combined_context: str
    agents_to_call: List[str]

workflow = StateGraph(GraphState)

workflow.add_node("Routingagent", Routingagent)
workflow.add_node("FOOD_AGENT", Foodagent)
workflow.add_node("CGM_AGENT", CGMagent)
workflow.add_node("IOT_AGENT", IOTagent)
workflow.add_node("MULTIOMICS_AGENT", MULTIOMICSagent)
workflow.add_node("combiner",combiner)

workflow.add_edge(START, "Routingagent")
workflow.add_conditional_edges(
    "Routingagent",
    lambda state: state["agents_to_call"],
    {
        "FOOD_AGENT": "FOOD_AGENT",
        "CGM_AGENT": "CGM_AGENT",
        "IOT_AGENT": "IOT_AGENT",
        "MULTIOMICS_AGENT": "MULTIOMICS_AGENT"
    }
)
workflow.add_node("join", lambda x: x)  # no-op join

workflow.add_edge("FOOD_AGENT", "join")
workflow.add_edge("CGM_AGENT", "join")
workflow.add_edge("IOT_AGENT", "join")
workflow.add_edge("MULTIOMICS_AGENT", "join")

workflow.add_edge("join", "combiner")
workflow.add_edge("combiner", END)
app = workflow.compile()




In [12]:
inputs = {"question": "summarize the Transcriptomic profiles of different cells in the patient."}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"Node '{key}':")
    print("\n---\n")

# Final generation
print(value["combined_context"])
print("--------------------------------------------------")
print("--------------------------------------------------")
print(value)

ValidationException: An error occurred (ValidationException) when calling the Converse operation: This model doesn't support tool use.